## Imports

In [ ]:
import os
from os.path import join
import sys
from pathlib import Path

# include app directory into sys.path
parent_dir = Path(os.path.abspath('')).parent
app_dir = join(parent_dir, "app")
if app_dir not in sys.path:
      sys.path.append(app_dir)

import torch as pt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker 
from flowtorch.analysis import SVD
import numpy as np
from scipy.fft import fft, fftfreq

import utils.config as config

# increase plot resolution
plt.rcParams["figure.dpi"] = 180

# use GPU if possible
device = pt.device("cuda") if pt.cuda.is_available() else pt.device("cpu")
print(device)

# define paths
DATA_PATH = join(parent_dir, "data", "SVD")
SVD_PATH = join(parent_dir, "output", "SVD")
OUTPUT_PATH = join(parent_dir, "output", "SVD")

TIMESTEP = config.timestep_reconstruction
TIMESTEP_dimless = (TIMESTEP * config.U_inf) / (config.c_mean * config.timesteps_per_second)

In [ ]:
# load test dataset
test_1 = pt.load(join(DATA_PATH, "X_test_1.pt"))
test_2 = pt.load(join(DATA_PATH, "X_test_2.pt"))

# compute temporal mean component for centering
test_1_mean = test_1.mean(dim=1).unsqueeze(-1)
test_2_mean = test_2.mean(dim=1).unsqueeze(-1)

# load coordinate grids
coords = pt.load(join(Path(DATA_PATH).parent, "coords_interp.pt"))
xx, yy = coords

In [ ]:
# load left singular vectors and temporal mean
U = pt.load(join(SVD_PATH, "U.pt"))
mean = pt.load(join(SVD_PATH, "mean.pt"))

# reconstruct datasets
test_1_reconstr = U[:,:config.SVD_rank] @ pt.transpose(U[:,:config.SVD_rank], 0, 1) @ (test_1 - mean) + mean
test_2_reconstr = U[:,:config.SVD_rank] @ pt.transpose(U[:,:config.SVD_rank], 0, 1) @ (test_2 - mean) + mean

In [ ]:
# reshape data for visualization
test_1 = test_1.unflatten(dim=0, sizes=config.target_resolution)
test_2 = test_2.unflatten(dim=0, sizes=config.target_resolution)

test_1_reconstr = test_1_reconstr.unflatten(dim=0, sizes=config.target_resolution)
test_2_reconstr = test_2_reconstr.unflatten(dim=0, sizes=config.target_resolution)

## Results for Test Flow Condition 1 ($\alpha = 3.00^\circ$)

#### Reconstruct $c_p$-snapshot of unseen flow condition

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
vmin_cp, vmax_cp = config.plot_lims_cp
vmin_MSE, vmax_MSE = config.plot_lims_MSE_reconstruction
levels_cp = pt.linspace(vmin_cp, vmax_cp, 120)
levels_MSE = pt.linspace(vmin_MSE, vmax_MSE, 120)

SE = (test_1[:,:,TIMESTEP] - test_1_reconstr[:,:,TIMESTEP])**2

ax1.contourf(xx, yy, test_1[:,:,TIMESTEP], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
ax2.contourf(xx, yy, test_1_reconstr[:,:,TIMESTEP], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
cont = ax3.contourf(xx, yy, SE, vmin=vmin_MSE, vmax=vmax_MSE, levels=levels_MSE)

ax1.set_title("Ground Truth")
ax2.set_title("SVD")

fig.subplots_adjust(right=0.95)
cax = fig.add_axes([0.99, 0.283, 0.03, 0.424])
cbar = fig.colorbar(cont, cax=cax,label = "Squarred Error")
cbar.formatter = ticker.FormatStrFormatter(f'%.{3}f')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect("equal")
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.savefig(join(OUTPUT_PATH, f"SVD_prediction_test_1.png"), bbox_inches="tight")

In [ ]:
# flatten original and reconstructed test dataset
test_1_original = test_1.flatten(0,1)
test_1_reconstr = test_1_reconstr.flatten(0,1)

#### Compare Power Spectra of POD Modes

In [ ]:
svd_original= SVD(test_1_original - test_1_original.mean(dim=1).unsqueeze(-1), rank=1e5)
V_original = svd_original.V

svd_reconstr = SVD(test_1_reconstr - test_1_reconstr.mean(dim=1).unsqueeze(-1), rank=1e5)
V_reconstr = svd_reconstr.V

N = test_1_original.shape[1]
num_modes = 4
sample_rate = 2000          # [Hz]
y_lims = [1e-7, 1e-1]

fig, ax = plt.subplots(2, 2, figsize=(6, 5), sharex=True)
for row in range(2):
    for col in range(2):
        # Calculate the mode index and retrieve mode coefficients
        mode = row * 2 + col                   
        original_mode_coeffs = V_original[:, mode].numpy()
        reconstr_mode_coeffs = V_reconstr[:, mode].numpy()

        # Compute FFT and PSD
        original_fft = fft(original_mode_coeffs)
        original_psd = np.abs(original_fft)**2 / len(original_fft)
        reconstr_fft = fft(reconstr_mode_coeffs)
        reconstr_psd = np.abs(reconstr_fft)**2 / len(reconstr_fft)

        # Frequency values for plotting
        freq = fftfreq(len(original_mode_coeffs), d=1/sample_rate)* config.c_mean * 0.5 / config.U_inf

        # Use only the positive frequencies (discard negative frequency half)
        freq = freq[:len(freq)//2]
        original_psd = original_psd[:len(original_psd)//2]
        reconstr_psd = reconstr_psd[:len(reconstr_psd)//2]

        # Plot the power spectra
        ax[row, col].semilogy(freq, original_psd, linewidth=0.5, color="black", label="Experimental Data")
        ax[row, col].semilogy(freq, reconstr_psd, linewidth=0.7, color="cornflowerblue", linestyle='dashed', label="SVD Reconstruction")
        ax[row, col].set_title(f"Mode Coefficient {mode + 1}")
        ax[row, col].grid()
        ax[row, col].set_yticklabels([])
        ax[row, col].set_yticks([])
        ax[row, col].set_ylim(y_lims)

        
ax[1, 0].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1, 1].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1,0].legend()

plt.xscale("log")
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, f"SVD_power_spectra_test_1.png"), bbox_inches="tight")

## Results for Test Flow Condition 1 ($\alpha = 5.00^\circ$)

#### Reconstruct $c_p$-snapshot of unseen flow condition

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
vmin_cp, vmax_cp = config.plot_lims_cp
vmin_MSE, vmax_MSE = config.plot_lims_MSE_reconstruction
levels_cp = pt.linspace(vmin_cp, vmax_cp, 120)
levels_MSE = pt.linspace(vmin_MSE, vmax_MSE, 120)

SE = (test_2[:,:,TIMESTEP] - test_2_reconstr[:,:,TIMESTEP])**2

ax1.contourf(xx, yy, test_2[:,:,TIMESTEP], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
ax2.contourf(xx, yy, test_2_reconstr[:,:,TIMESTEP], vmin=vmin_cp, vmax=vmax_cp, levels=levels_cp)
cont = ax3.contourf(xx, yy, SE, vmin=vmin_MSE, vmax=vmax_MSE, levels=levels_MSE)

ax1.set_title("Ground Truth")
ax2.set_title("SVD")

fig.subplots_adjust(right=0.95)
cax = fig.add_axes([0.99, 0.283, 0.03, 0.424])
cbar = fig.colorbar(cont, cax=cax,label = "Squarred Error")
cbar.formatter = ticker.FormatStrFormatter(f'%.{3}f')

for ax in [ax1, ax2, ax3]:
    ax.set_aspect("equal")
    ax.set_xticklabels([])
    ax.set_yticklabels([])

fig.savefig(join(OUTPUT_PATH, f"SVD_prediction_test_2.png"), bbox_inches="tight")

In [ ]:
# flatten original and reconstructed test dataset
test_2_original = test_2.flatten(0,1)
test_2_reconstr = test_2_reconstr.flatten(0,1)

#### Compare Power Spectra of POD Modes

In [ ]:
svd_original= SVD(test_2_original - test_2_original.mean(dim=1).unsqueeze(-1), rank=1e5)
V_original = svd_original.V

svd_reconstr = SVD(test_2_reconstr - test_2_reconstr.mean(dim=1).unsqueeze(-1), rank=1e5)
V_reconstr = svd_reconstr.V

N = test_2_original.shape[1]
num_modes = 4
sample_rate = 2000          # [Hz]

fig, ax = plt.subplots(2, 2, figsize=(6, 5), sharex=True)
for row in range(2):
    for col in range(2):
        # Calculate the mode index and retrieve mode coefficients
        mode = row * 2 + col                   
        original_mode_coeffs = V_original[:, mode].numpy()
        reconstr_mode_coeffs = V_reconstr[:, mode].numpy()

        # Compute FFT and PSD
        original_fft = fft(original_mode_coeffs)
        original_psd = np.abs(original_fft)**2 / len(original_fft)
        reconstr_fft = fft(reconstr_mode_coeffs)
        reconstr_psd = np.abs(reconstr_fft)**2 / len(reconstr_fft)

        # Frequency values for plotting
        freq = fftfreq(len(original_mode_coeffs), d=1/sample_rate)* config.c_mean * 0.5 / config.U_inf

        # Use only the positive frequencies (discard negative frequency half)
        freq = freq[:len(freq)//2]
        original_psd = original_psd[:len(original_psd)//2]
        reconstr_psd = reconstr_psd[:len(reconstr_psd)//2]

        # Plot the power spectra
        ax[row, col].semilogy(freq, original_psd, linewidth=0.5, color="black", label="Experimental Data")
        ax[row, col].semilogy(freq, reconstr_psd, linewidth=0.7, color="cornflowerblue", linestyle='dashed', label="SVD Reconstruction")
        ax[row, col].set_title(f"Mode Coefficient {mode + 1}")
        ax[row, col].grid()
        ax[row, col].set_yticklabels([])
        ax[row, col].set_yticks([])
        ax[row, col].set_ylim(y_lims)

        
ax[1, 0].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1, 1].set_xlabel(rf"Reduced Frequency $\omega$")
ax[1,0].legend()

plt.xscale("log")
fig.tight_layout()
fig.savefig(join(OUTPUT_PATH, f"SVD_power_spectra_test_2.png"), bbox_inches="tight")